## Evaluate ESG scores for bonds portfolio

In [16]:
# load imports
import pandas as pd
import json, math
import plotly.graph_objects as go

In [2]:
# execute the helper functions defined for accessing RDP REST API calls 
%run RDPDefines.ipynb

### What is Symbology Mapping

Symbology Endpoint: [api.refinitiv.com/discovery/symbology/v1/lookup]()

parameters [route = FindESGStatementParent]()

<br/>
Examples:
<br/>
<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 44654483026        |         |
| Issuer     | SAUDI ELECTRICITY GLOBAL SUKUK COMPANY 3        |         |
| 1st Parent | 4295887339 (Saudi Electricity Company)        |    ✔     |
| Result     | 4295887339        |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 192814833479        |         |
| Issuer     | GOLDMAN SACHS FINANCE CORP INTERNATIONAL LTD        |         |
| 1st Parent | GS Global Markets Inc        |         |
| 2nd Parent | 4295911963 (Goldman Sachs Group Inc)        |    ✔   |
| Result     | 4295911963       |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 192846098875        |         |
| Issuer     | MORGAN STANLEY BANK NA        |         |
| 1st Parent | MORGAN STANLEY DOMESTIC HOLDINGS INC        |         |
| 2nd Parent | Morgan Stanley Capital Management LLC        |         |
| 3rd Parent | 4295904557 (Morgan Stanley)        |    ✔   |
| Result     | 4295904557        |         |

<br/>

|            | Name    | Has ESG |
|------------|---------|---------|
| Bond       | 46641173275        |         |
| Issuer     | PROPERTY AND BUILDING CORP LTD        |         |
| 1st Parent | DISCOUNT INVESTMENT CORP LTD        |         |
| 2nd Parent | DOLPHIN NETHERLANDS BV        |         |
| 3rd Parent | TYRUS SA        |         |
| 4th Parent | 5000620306 (RSA INVERSIONES Y REPRESENTACIONES SA)        |    ✔    |
| Result     | 5000620306        |         |


### Download the bulk JSON files

In [3]:
# define the download function
def downloadJSONBulkFile(bucketName, fileAttributes, fileNameKeywords):
    # get a list of all the buckets
    hResp = getRequest('/file-store/v1/file-sets?bucket=' + bucketName + '&pageSize=100&attributes=' + fileAttributes)
    print(hResp)
    # loop through all the buckets
    for bucket in hResp['value']:
        bName = bucket['name']
        # does bucket contains all the matching keywords
        if all([x in bName for x in fileNameKeywords]):
            fileName = bucket['files'][0]
            print('Found bucket: ', bName, ', FileName: ', fileName)
            # stop any more searching
            break
    
    if not fileName:
        raise Exception('No matching bulk file found in bucket:'.format(bucketName))

    # download and uncompress the file object
    fileStr = downloadUncompressFile('/file-store/v1/files/' + fileName + '/stream')
    print('File downloaded and uncompressed, size: ', len(fileStr))
    return fileStr


#### Download and save the Bond ISIN - ESG Parent mapping

In [4]:
# download the Bond-ESGParent symbology database
jsonlFile = downloadJSONBulkFile('bulk-symbology', 'ContentType:Symbology BondISINSusFinMapping', ['Bond', 'ISIN', 'Json', 'Init'])
# parse out the entries in the bulk file
mapping = []
for l in jsonlFile.splitlines():
    jObj = json.loads(l)
    if len(jObj['Identifiers']) > 0 and jObj['EsgCoverage']['EsgStatementParentOrganization']:
        coName = jObj['EsgCoverage']['EsgStatementParentOrganization']['PartyName']['Names'][0]['NormalizedName'] if jObj['EsgCoverage']['EsgStatementParentOrganization']['PartyName']['Names'] else ''
        mapping.append((jObj['Identifiers'][0]['IdentifierValue'], jObj['EsgCoverage']['EsgStatementParentOrganization']['ObjectId'], coName))
    
print('Loaded {} Bonds ISIN to ESG Parent PermID mappings'.format(len(mapping)))

Getting access token...
...token received
{'value': [{'id': '4018-d7c1-b8574d07-bd57-ae61e105c79f', 'name': 'Bulk-Global-BondISINSusFinMapping-v1-Csv-Init-2023-05-07T16:01:06.616Z', 'bucketName': 'bulk-Symbology', 'packageId': '4edd-8742-9ee36c59-af01-9d2166299ba6', 'attributes': [{'name': 'ContentType', 'value': 'Symbology BondISINSusFinMapping'}], 'files': ['4122-50bf-eadac63b-a83e-42af614ea901', '4331-df2d-f9f8e2d1-80ef-c817c86fea49', '4384-0ffd-a981ba8f-bf8d-44d37482573f', '4d2e-14d0-c00da7f9-8952-54b4c83b10ad', '4e2a-9d4b-1ac03f10-bc81-5488cf3a7f16'], 'numFiles': 5, 'contentFrom': '1970-01-01T00:00:00Z', 'contentTo': '2023-05-07T15:55:00Z', 'availableFrom': '2023-05-07T17:03:42Z', 'availableTo': '2023-08-07T17:03:42Z', 'status': 'READY', 'created': '2023-05-07T17:03:42Z', 'modified': '2023-05-07T17:04:09Z'}, {'id': '40ee-6a26-720c2490-8d1b-9382fa738ffa', 'name': 'Bulk-Global-BondISINSusFinMapping-v1-Csv-Init-2023-04-23T16:01:49.957Z', 'bucketName': 'bulk-Symbology', 'packageId': '

In [5]:
# load the dataset into a pandas dataframe
df1 = pd.DataFrame(mapping, columns=['Bond', 'ESGParent', 'ParentName'])
# save the database
df1.to_pickle('Bond_Parent_mapping.pkl')

#### Download and save the ESG Scores dataset

In [6]:
# download the ESG Scores database
jsonlFile = downloadJSONBulkFile('ESG', 'ContentType:ESG Scores', ['Scores', 'Full', 'Init'])

{'value': [{'id': '4073-5412-0dda9817-ab0d-cdbfe17aec94', 'name': 'RFT-ESG-Scores-Full-Delta-2023-04-30', 'bucketName': 'ESG', 'packageId': '42de-14b7-37470ec8-9087-ccd1a1bae75d', 'attributes': [{'name': 'ResultCount', 'value': '56829'}, {'name': 'ContentType', 'value': 'ESG Scores'}], 'files': ['4a41-adc4-5e74aa70-9ac8-0283cd9810f3'], 'numFiles': 1, 'contentFrom': '2023-04-23T17:51:16Z', 'contentTo': '2023-04-30T16:25:00Z', 'availableFrom': '2023-04-30T16:36:14Z', 'availableTo': '2023-05-14T16:36:14Z', 'status': 'READY', 'created': '2023-04-30T16:36:14Z', 'modified': '2023-04-30T16:36:25Z'}, {'id': '40aa-7545-dd4034ad-9f40-47598e907ba8', 'name': 'RFT-ESG-Scores-Wealth-Standard-delta-2023-05-07', 'bucketName': 'ESG', 'packageId': '4bcc-4602-0a57ebb2-baf0-1fc9825e76b6', 'attributes': [{'name': 'ContentType', 'value': 'ESG Scores'}, {'name': 'ResultCount', 'value': '33761'}], 'files': ['4779-3355-f8d6d39c-a3b7-1ebaef702805'], 'numFiles': 1, 'contentFrom': '2022-11-06T17:15:00Z', 'content

In [7]:
scores = []
for l in jsonlFile.splitlines():
    j = json.loads(l)
    e = j['ESGScores']
    scores.append((j['StatementDetails']['OrganizationId'],
        j['StatementDetails']['FinancialPeriodFiscalYear'],
        e['ESGCombinedScore']['Value'], 
        e['ESGScore']['Value'],
        e['EnvironmentPillarScore']['Value'],
        e['ESGResourceUseScore']['Value'],
        e['ESGEmissionsScore']['Value'],
        e['ESGInnovationScore']['Value'],
        e['SocialPillarScore']['Value'],
        e['ESGWorkforceScore']['Value'],
        e['ESGHumanRightsScore']['Value'],
        e['ESGCommunityScore']['Value'],
        e['ESGProductResponsibilityScore']['Value'],
        e['GovernancePillarScore']['Value'],
        e['ESGManagementScore']['Value'],
        e['ESGShareholdersScore']['Value'],
        e['ESGCsrStrategyScore']['Value'],
        e['ESGCControversiesScore']['Value']))


print('Loaded {} scores'.format(len(scores))) 


Loaded 104861 scores


In [8]:
# load the dataset into a pandas dataframe
df2 = pd.DataFrame(scores, columns=['OrganizationId', 'FiscalYear', 'ESGCombinedScore', 'ESGScore', 'EnvironmentPillarScore', 'ESGResourceUseScore', 'ESGEmissionsScore', 'ESGInnovationScore', 'SocialPillarScore', 'ESGWorkforceScore', 'ESGHumanRightsScore', 'ESGCommunityScore', 'ESGProductResponsibilityScore', 'GovernancePillarScore', 'ESGManagementScore', 'ESGShareholdersScore', 'ESGCsrStrategyScore', 'ESGCControversiesScore'])
# change the Fiscal Year data type to a number
df2['FiscalYear'] = df2['FiscalYear'].astype(int)
# keep the latest ESG scores only
df2 = df2.loc[df2.groupby(['OrganizationId'])['FiscalYear'].idxmax()].reset_index(drop=True)
# save the database
df2.to_pickle('ESGScores.pkl')

### Load the pre-downloaded database for Symbology mapping and ESG

In [3]:
bMapping = pd.read_pickle('Bond_Parent_mapping.pkl')
bMapping.head()

,Bond,ESGParent,ParentName
0,KR6HN0001YR8,4295882718,Hana Financial Group Inc
1,KR6703304A47,4295882718,Hana Financial Group Inc
2,US78016FGF53,8589934213,Royal Bank of Canada
3,XS2291434251,4297375292,Marex Financial
4,XS2531033020,4295888106,DBS Group Holdings Ltd


In [40]:
scores = pd.read_pickle('ESGScores.pkl').astype({'ESGCombinedScore': float, 'ESGScore': float, 'EnvironmentPillarScore': float, 'ESGResourceUseScore': float, 'ESGEmissionsScore': float, 'ESGInnovationScore': float, 'SocialPillarScore': float, 'ESGWorkforceScore': float, 'ESGHumanRightsScore': float, 'ESGCommunityScore': float, 'ESGProductResponsibilityScore': float, 'GovernancePillarScore': float, 'ESGManagementScore': float, 'ESGShareholdersScore': float, 'ESGCsrStrategyScore': float, 'ESGCControversiesScore': float})
scores.head()

,OrganizationId,FiscalYear,ESGCombinedScore,ESGScore,EnvironmentPillarScore,ESGResourceUseScore,ESGEmissionsScore,ESGInnovationScore,SocialPillarScore,ESGWorkforceScore,ESGHumanRightsScore,ESGCommunityScore,ESGProductResponsibilityScore,GovernancePillarScore,ESGManagementScore,ESGShareholdersScore,ESGCsrStrategyScore,ESGCControversiesScore
0,4295533401,2019,0.340828,0.340828,0.041370,0.075092,0.073529,0.000000,0.544355,0.345261,0.742358,0.696325,0.351293,0.255405,0.301104,0.273344,0.000000,1.0
1,4295613014,2019,0.413583,0.413583,0.603543,0.818452,0.497340,0.567308,0.478389,0.345041,0.571970,0.663223,0.360515,0.274655,0.159367,0.203163,0.958333,1.0
2,4295641240,2022,0.648443,0.648443,0.444813,0.761307,0.694444,0.000000,0.838232,0.618721,0.832418,0.970320,0.969048,0.579649,0.468420,0.967413,0.554149,1.0
3,4295856018,2021,0.164220,0.164220,0.088141,0.000000,0.192308,0.000000,0.203273,0.221875,0.107477,0.587500,0.000000,0.162817,0.153226,0.250000,0.080000,1.0
4,4295856019,2021,0.128668,0.128668,0.000000,0.000000,0.000000,0.000000,0.128514,0.007937,0.000000,0.009524,0.832155,0.317656,0.201613,0.862903,0.080000,1.0


### Get the Bond portfolio holdings

In [5]:
# what is the Lipper ID of the bonds portfolio
portfolioID = 60000170

In [6]:
# get the constituents bonds in this portfolio
hResp = getRequest('/data/funds/v1/assets/' + str(portfolioID), {'properties': 'holdings'})
print(hResp)

Getting access token...
...token received
{'assets': [{'id': '60000170', 'holdings': [{'date': '2023-03-31', 'securitiesHeldCount': 1174, 'constituents': [{'name': 'UNITED STATES OF AMERICA 6.25% 15-MAY-2030', 'country': 'UNITED STATES', 'weight': 4.6235, 'weightPrevious': 4.5477, 'weightChange': 0.0758, 'type': {'id': '13737', 'code': 'SOVEREIGN BOND', 'name': 'Sovereign Bond'}, 'sharesHeld': 747077200.0, 'sharesPrevious': 747077200.0, 'sharesChange': 0.0, 'crossReferenceCodes': [{'code': 'RIC', 'type': {'id': '26', 'code': 'RIC', 'name': 'RIC'}, 'values': [{'value': '912810FM5='}]}, {'code': 'ISIN', 'type': {'id': '1424', 'code': 'ISIN', 'name': 'ISIN Code'}, 'values': [{'value': 'US912810FM54'}]}, {'code': 'CUSIP', 'type': {'id': '4862', 'code': 'CUSIP', 'name': 'CUSIP'}, 'values': [{'value': '912810FM5'}]}, {'code': 'SEDOL', 'type': {'id': '1420', 'code': 'SEDOL', 'name': 'Sedol Code'}, 'values': [{'value': 'B7T79C5'}]}], 'rank': 1, 'marketValue': 870928554.7, 'marketValueCurrency'

In [7]:
allHoldings = []
# extract the ISIN, and weights of the bond holdings
for a in hResp['assets'][0]['holdings'][0]['constituents']:
    if 'crossReferenceCodes' in a:
        for code in a['crossReferenceCodes']:
            if code['code'] == 'ISIN':
                allHoldings.append((code['values'][0]['value'], a['weight']))

display(allHoldings[:10])
print('This fund contains {} bonds'.format(len(allHoldings)))

[('US912810FM54', 4.6235),
 ('US912810FJ26', 3.2488),
 ('US91282CGP05', 2.5624),
 ('US91282CGH88', 2.5565),
 ('US31359MGK36', 2.2565),
 ('US912810FB99', 2.0138),
 ('US91282CFV81', 1.7113),
 ('LU1900232734', 1.4823),
 ('US912810ET17', 1.223),
 ('US21H0306413', 1.0868)]

This fund contains 1163 bonds


### Match the ESG-Parent company of these bonds

In [41]:
# create a master dataframe for all processing
mdf = pd.DataFrame(allHoldings, columns =['Bond', 'Weight'])
# merge the ESG parent company info into this dataframe
mdf = mdf.merge(bMapping, how='left', left_on='Bond', right_on='Bond')
display(mdf)

,Bond,Weight,ESGParent,ParentName
0,US912810FM54,4.6235,NaN,NaN
1,US912810FJ26,3.2488,NaN,NaN
2,US91282CGP05,2.5624,NaN,NaN
3,US91282CGH88,2.5565,NaN,NaN
4,US31359MGK36,2.2565,4295903973,Federal National Mortgage Association
...,...,...,...,...
1158,US12591RBB50,0.0001,NaN,NaN
1159,XS2406881669,0.0000,4295865535,Yango Group Co Ltd
1160,USL9116PAG83,0.0000,NaN,NaN
1161,US92824BAA44,0.0000,NaN,NaN


In [42]:
total = len(allHoldings)
covered = len(mdf['ESGParent'].dropna())
coverage = (covered / total) * 100
fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    value = coverage,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': 'Coverage % ({} out of {} have ESG data)'.format(covered, total) },
    gauge = {'axis': {'range': [None, 100]}}))

fig.show()

### Calculate and display the consolidated ESG Score for the whole portfolio

In [52]:
# formulate everything onto a dataframe and display
combined = mdf.merge(scores, how='left', left_on='ESGParent', right_on='OrganizationId')
combined.drop('OrganizationId', axis=1, inplace=True)
display(combined)

,Bond,Weight,ESGParent,ParentName,FiscalYear,ESGCombinedScore,ESGScore,EnvironmentPillarScore,ESGResourceUseScore,ESGEmissionsScore,ESGInnovationScore,SocialPillarScore,ESGWorkforceScore,ESGHumanRightsScore,ESGCommunityScore,ESGProductResponsibilityScore,GovernancePillarScore,ESGManagementScore,ESGShareholdersScore,ESGCsrStrategyScore,ESGCControversiesScore
0,US912810FM54,4.62e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US912810FJ26,3.25e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US91282CGP05,2.56e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US91282CGH88,2.56e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US31359MGK36,2.26e+00,4295903973,Federal National Mortgage Association,2022.0,0.54,0.54,0.44,0.27,0.10,0.56,0.50,0.58,0.0,0.61,0.76,0.64,0.85,0.30,0.07,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1158,US12591RBB50,1.00e-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1159,XS2406881669,0.00e+00,4295865535,Yango Group Co Ltd,2021.0,0.46,0.46,0.39,0.48,0.11,0.66,0.29,0.26,0.2,0.16,0.78,0.73,0.75,0.91,0.36,1.0
1160,USL9116PAG83,0.00e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1161,US92824BAA44,0.00e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Rebase, calculate the combined ESG scores of these holdings
weightedSeries = []
for idx, a in combined['ESGCombinedScore'].items():
    if math.isnan(a):
        weightedSeries.append(0)
    else:
        weightedSeries.append(combined['Weight'][idx])

weightTotal = sum(weightedSeries)
rebasedWeight = combined['Weight']/weightTotal

In [56]:
# calculate the weighted total for the holdings
total = []
for col in combined:
    if col == 'Bond':
        total.append('WEIGHTED AVERAGE')
    elif col == 'Weight':
        total.append(1.0)
    elif col == 'FiscalYear':
        total.append('')
    elif combined[col].dtype == 'float64':
        total.append((combined[col] * rebasedWeight).sum())
    else:
        total.append('')

In [58]:
# insert the final result into the portfolio
combined.loc[-1] = total
combined.index = combined.index + 1
combined = combined.sort_index()

In [59]:
# display the final dataframe
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option("display.precision", 2)
display(combined.fillna(''))

,Bond,Weight,ESGParent,ParentName,FiscalYear,ESGCombinedScore,ESGScore,EnvironmentPillarScore,ESGResourceUseScore,ESGEmissionsScore,ESGInnovationScore,SocialPillarScore,ESGWorkforceScore,ESGHumanRightsScore,ESGCommunityScore,ESGProductResponsibilityScore,GovernancePillarScore,ESGManagementScore,ESGShareholdersScore,ESGCsrStrategyScore,ESGCControversiesScore
0,WEIGHTED AVERAGE,1.00e+00,,,,0.53,0.63,0.57,0.61,0.61,0.42,0.64,0.69,0.53,0.74,0.57,0.65,0.68,0.57,0.59,0.68
1,US912810FM54,4.62e+00,,,,,,,,,,,,,,,,,,,
2,US912810FJ26,3.25e+00,,,,,,,,,,,,,,,,,,,
3,US91282CGP05,2.56e+00,,,,,,,,,,,,,,,,,,,
4,US91282CGH88,2.56e+00,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159,US12591RBB50,1.00e-04,,,,,,,,,,,,,,,,,,,
1160,XS2406881669,0.00e+00,4295865535,Yango Group Co Ltd,2021.0,0.46,0.46,0.39,0.48,0.11,0.66,0.29,0.26,0.2,0.16,0.78,0.73,0.75,0.91,0.36,1.0
1161,USL9116PAG83,0.00e+00,,,,,,,,,,,,,,,,,,,
1162,US92824BAA44,0.00e+00,,,,,,,,,,,,,,,,,,,


In [ ]:
combined.to_clipboard()